# Build Data - Other Problems

In [1]:
import numpy as np
import pandas as pd

from datetime import date
from datetime import datetime

import sys
sys.path.insert(0, '../cmds')
from ficcvol import *
from binomial import *
from ratecurves import *
from treasury_cmds import compound_rate
from volskew import *

import datetime

In [2]:
DATE = '2025-03-10'

In [3]:
SAVEDATA = True
FILEOUT = f'../data/fiderivs_{DATE}.xlsx'

# Curves

In [4]:
freqcurve = 4
ROUND = 4
curves = pd.read_excel(f'../data/cap_curves_{DATE}.xlsx', sheet_name=f'rate curves {DATE}').set_index('tenor')

### Limit maturity

In [5]:
LIMIT = 5
curves = curves.loc[:LIMIT]

# BDT

In [6]:
sigmas = curves['fwd vols']
sigmas.iloc[0] = sigmas.iloc[1]
sigmas.index = sigmas.index.to_numpy().round(6)

### Simple BDT (constant vol)

In [7]:
# theta, ratetree = estimate_theta(sigmas,100*curves.loc[:,'discounts'])
# format_bintree(ratetree,style='{:.2%}')

### New Clean Code

### Non-uniform grid

In [8]:
# sys.path.insert(0, '../dev')
# from bdt_nonuni import *

# T = 5
# state, ratetree = estimate_topnode_nonuniform(curves.loc[:T,'fwd vols'],curves.loc[:T,'discounts'],round_digits=ROUND)

### Working Code

In [9]:
T = 5
dt = curves.index[0]
state, ratetree = estimate_topnode(curves.loc[:T+dt,'fwd vols'],curves.loc[:T+dt,'discounts'],round_digits=ROUND)
#format_bintree(ratetree,style='{:.2%}')

0.25
0.5
0.75
1.0
1.25
1.5
1.75
2.0
2.25
2.5
2.75
3.0
3.25
3.5
3.75
4.0
4.25
4.5
4.75


In [10]:
ratetree_alt = ratetree.copy()
ratetree.columns = np.round(ratetree.columns,2)

curves_alt = curves.copy()
curves.index = np.round(curves.index,2)

### Extract Fwd Vol from Flat Vol via Tree?

In [11]:
def flatvol_to_capfloor_prices(discounts,forward_rates,swaprates,flatvols,Tmax,dt=.25,N=100):
    
    maturities = np.arange(dt,Tmax+dt,dt)
    capfloor = pd.DataFrame(index=maturities, columns=['cap','floor'])

    for T in maturities:

        strike = swaprates.loc[T]
        
        cflets = pd.DataFrame(index=capfloor.loc[:T].index.to_numpy(),columns=['caplet','floorlet'])

        for i,Tval in enumerate(cflets.index):
            if i==0:
                cflets.loc[Tval,:] = 0
            else:
                fwdrate = forward_rates.loc[Tval]
                cflets.loc[Tval,'caplet'] = N * (1/dt) * blacks_formula(Tval-1/dt,flatvols.loc[Tval],strike,fwdrate,discounts.loc[Tval],isCall=True)
                cflets.loc[Tval,'floorlet'] = N * (1/dt) * blacks_formula(Tval-1/dt,flatvols.loc[Tval],strike,fwdrate,discounts.loc[Tval],isCall=False)


        capfloor.loc[T,'cap'] = cflets['caplet'].sum()
        capfloor.loc[T,'floor'] = cflets['floorlet'].sum()

    return capfloor

In [12]:
# flatvols = curves['flat vols']
# discounts = curves['discounts']
# forward_rates = curves['forwards']
# swaprates = curves['swap rates']

# capfloor_px = flatvol_to_capfloor_prices(discounts,forward_rates,swaprates,flatvols,Tmax=T)
# cap_prices = capfloor_px['cap']

In [13]:
# maturities = np.arange(dt,T+dt,dt)
# caps = pd.DataFrame(index=maturities, columns=['price'])
# for t in maturities:
#     caps.loc[t,'price'] = cap_vol_to_price(curves.loc[t,'flat vols'], curves.loc[t,'swap rates'], curves.loc[:t,'forwards'], curves.loc[:t,'discounts'], dt=.25, notional=100)

In [14]:
# state, rtree = estimate_topnode_vol(curves.loc[:T+dt,'discounts'],caps['price'],curves.loc[:T+dt,'swap rates'])
# format_bintree(rtree,style='{:.2%}')

***

# Vol Skew

These are input manually from Bloomberg's `VCUB` dashboard.

In [15]:
voldata = pd.read_excel(f'../data/swaption_vol_data_{DATE}.xlsx',sheet_name='select')
voldata

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
0,SOFR,swaption,black,2025-03-10,1,3,51.89,37.53,33.24,31.65,30.41,29.48,28.84,28.25,28.64


In [16]:
IDVOLSKEW = 0
voldata = voldata.loc[[IDVOLSKEW],:]

# SABR Fit

In [17]:
SWAP_TYPE = 'SOFR'
QUOTE_STYLE = 'black'
RELATIVE_STRIKE = 0

expry = 1
tenor = 3
# expry_round = 1
# tenor_round = 4

# index_array = ratetree.columns.to_numpy()

# closest_index = index_array[np.abs(index_array - tenor_round).argmin()]
# tenor = closest_index

# closest_index = index_array[np.abs(index_array - expry_round).argmin()]
# expry = closest_index

freqswap = 4

Topt = expry
Tswap = Topt+tenor
fwdswap = calc_fwdswaprate(curves['discounts'], Topt, Tswap, freqswap=freqswap)

In [18]:
doSLIM = False
BETA = 0.25
F = fwdswap

In [19]:
volquote = voldata.query(f'model=="{QUOTE_STYLE}"').query(f'reference=="{SWAP_TYPE}"').query(f'date=="{DATE}"')#.query(f'expiration=="{expry}"').query(f'tenor=="{tenor}"')
idx = (volquote['expiration']==expry) & (volquote['tenor']==tenor)
volquote = volquote.loc[idx]
volquote.index = ['implied vol']

strikerange = np.array([col for col in volquote.columns if isinstance(col, int)])

vols = volquote[strikerange]
vols /= 100
strikes = fwdswap + strikerange/100/100
idstrike = np.where(strikerange==RELATIVE_STRIKE)[0][0]

idstrikeATM = np.where(strikerange==0)[0][0]

if QUOTE_STYLE == 'normal':
    vols /= 100 * fwdrate

capvol = curves.loc[Topt,'fwd vols']

strikeATM = strikes[idstrikeATM]
volATM = vols.iloc[0,idstrikeATM]

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_6982/716694739.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  volquote = voldata.query(f'model=="{QUOTE_STYLE}"').query(f'reference=="{SWAP_TYPE}"').query(f'date=="{DATE}"')#.query(f'expiration=="{expry}"').query(f'tenor=="{tenor}"')


In [20]:
period_fwd = curves.index.get_loc(Topt)
period_swap = curves.index.get_loc(Tswap)+1
step = round(freqcurve/freqswap)

discount = curves['discounts'].iloc[period_fwd+step : period_swap : step].sum()/freqswap
blacks_quotes = vols.copy()
blacks_quotes.loc['strike'] = strikes
blacks_quotes = blacks_quotes.loc[['strike','implied vol']]

blacks_quotes.style.format('{:.4f}')

,-200,-100,-50,-25,0,25,50,100,200
strike,0.0150,0.0250,0.0300,0.0325,0.0350,0.0375,0.0400,0.0450,0.0550
implied vol,0.5189,0.3753,0.3324,0.3165,0.3041,0.2948,0.2884,0.2825,0.2864


#### Limit SABR fit to quotes within +/- 200bps

In [21]:
#vols = vols.loc[['implied vol'],-200:200]
#strikes = strikes[1:-1]

In [22]:
def obj_fun(xargs):
    nu = xargs[0]
    rho = xargs[1]
    alpha = xargs[2]
    
    ivolSABR = np.zeros(len(strikes))
    
    for i,strike in enumerate(strikes):
         ivolSABR[i] = sabr(BETA,nu,rho,alpha,F,strike,Topt)
    
    error = ((ivolSABR - vols.values)**2).sum()
    
    return error


def obj_fun_slim(xargs):
    nu = xargs[0]
    rho = xargs[1]
    ivolSABR = np.zeros(len(strikes))
    
    for i,strike in enumerate(strikes):
         ivolSABR[i] = sabr_slim(BETA,nu,rho,F,strike,Topt,volATM)
    
    error = ((ivolSABR - vols.values)**2).sum()
    
    return error

In [23]:
if not doSLIM:
    x0 = np.array([.6,0,.1])
    fun = obj_fun
else:
    fun = obj_fun_slim
    x0 = np.array([.6,0,.1])

optim = minimize(fun,x0)
xstar = optim.x
nustar = xstar[0]
rhostar = xstar[1]

if doSLIM:
    alphastar = solve_alpha(BETA,nustar,rhostar,Topt,volATM,F)
    ivolSABR = sabr_slim(BETA,nustar,rhostar,F,strikes,Topt,volATM)
else:
    alphastar = xstar[2]
    ivolSABR = sabr(BETA,nustar,rhostar,alphastar,F,strikes,Topt)
    
error = optim.fun

/Users/markhendricks/Projects/finm-fiderivs-2025/build_data/../cmds/volskew.py:77: RuntimeWarning: invalid value encountered in scalar divide
  sigmaB = (NUM/DEN) * (z/chi(z,rho))
/Users/markhendricks/Projects/finm-fiderivs-2025/build_data/../cmds/volskew.py:77: RuntimeWarning: invalid value encountered in divide
  sigmaB = (NUM/DEN) * (z/chi(z,rho))


In [24]:
sabrparams = pd.DataFrame([BETA,alphastar,nustar,rhostar],index=['beta','alpha','nu','rho'],columns=['estimate']).style.format('{:.4f}')
sabrparams.index.name = 'parameter'

### Save Data

In [25]:
if SAVEDATA:    
    with pd.ExcelWriter(FILEOUT) as writer:  
        curves.to_excel(writer, sheet_name= f'rate curves', index=True)
        ratetree.to_excel(writer, sheet_name= f'rate tree', index=True)
        voldata.to_excel(writer, sheet_name= 'bloomberg vcub', index=False)
        sabrparams.to_excel(writer, sheet_name='sabr params', index=True)